In [1]:
import numpy as np   # Downloading numpy for array format
import pandas as pd  # Downloading pandas for data import
import matplotlib.pyplot as plt # Matplotlib.pyplot for visualization
import seaborn as sns  # Visualisation
%matplotlib inline 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 1362, in _path_importer_cache
KeyError: 'C:\\Users\\paddy\\anaconda3\\lib\\site-packages\\numpy\\matrixlib'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\paddy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\paddy\AppData\Local\Temp\ipykernel_21288\1353763912.py", line 1, in <module>
    import numpy as np   # Downloading numpy for array format
  File "C:\Users\paddy\anaconda3\lib\site-packages\numpy\__init__.py", line 143, in <module>
    from . import lib
  File "C:\Users\paddy\anaconda3\lib\site-packages\numpy\lib\__init__.py", line 25, in <module>
    from .index_tricks import *
  File "C:\Users\paddy\anaconda3\lib\site-packages\numpy\lib\index_tricks.py", line 12, in <module>
    import numpy.matrixlib as matrix

TypeError: object of type 'NoneType' has no len()

In [ ]:
oil_prices=pd.read_csv('crude_oil_import_prices.csv') # importing the dataset

oil_prices.head(10)

In [ ]:
oil_prices.shape # Getting shape  of data

In [ ]:
oil_prices.info()   # checking nulls and data types

In [ ]:
oil_prices.dtypes

In [ ]:
oil_prices=oil_prices.drop(['INDICATOR', 'SUBJECT', 'MEASURE', 'Flag Codes'], axis=1)

oil_prices.head(5)

In [ ]:
oil_prices.dtypes

In [ ]:
oil_prices.shape

In [ ]:
oil_prices.describe()     # Basic statistical info - seems to skew to the left

In [ ]:
oil_prices=oil_prices.sort_values(by='TIME') # sort the data by year

oil_prices.head(10)
oil_prices.tail(5)

In [ ]:
oil_prices['TIME']=oil_prices['TIME'].str.slice(0, 4) # Removed extra characters from time column

oil_prices.tail(5)


In [ ]:
oil_prices.count()  # checking balance of observations between columns

In [ ]:
oil_prices.shape # Checkpoint on shape

In [ ]:
#duplicate_rows=oil_prices[oil_prices.duplicated()]

#print("number of duplicate rows:", duplicate_rows.shape)

In [ ]:
print(oil_prices.isnull().sum())  # Looking for null values

In [ ]:
value_column_row_null= oil_prices[oil_prices['Value'].isnull()] # Seeing if any pattern in the null values

value_column_row_null.head(100)

In [ ]:
oil_prices=oil_prices.dropna().reset_index(drop=True)

oil_prices.count()

In [ ]:
oil_prices.shape

In [ ]:
oil_prices.head(15)

In [ ]:
oil_prices= oil_prices[oil_prices['FREQUENCY']=='M'].reset_index(drop=True) 

oil_prices.head(20)      # removed all data that wasn't monthly for consistent interval frequency

In [ ]:
sns.boxplot(x=oil_prices['Value'])      # view of the data which is skewed to the left

In [ ]:
oil_prices=oil_prices.sort_values(by=['TIME', 'LOCATION']).reset_index(drop=True)

In [ ]:
oil_prices.tail(50)   # Data now sorted by year and alphabetically by countries within each year for trend

In [ ]:
country_counts=oil_prices.groupby(['LOCATION', 'TIME']).size().reset_index(name='Count')

In [ ]:
country_counts.head(50)  # counting how many months are recorded per country in each year for data consistency

In [ ]:
oil_prices=oil_prices.merge(country_counts, on=['LOCATION', 'TIME'], how='left') # merging country_counts to oil_prices

In [ ]:
oil_prices=pd.DataFrame(oil_prices)  #Ensuring is a dataframe format

In [ ]:
oil_prices.head(5) # Checking the merge was successful

In [ ]:
oil_prices.tail(5)

In [ ]:
oil_prices=oil_prices[oil_prices['Count']==12] # Filtering for only TIME/LOCATION combinations with full 12 months worth of data

In [ ]:
oil_prices.head(15)

In [ ]:
oil_prices.tail(20)

In [ ]:
oil_prices=oil_prices.drop(['FREQUENCY', 'Count'], axis=1) # Dropping unnecessary columns

oil_prices.head(5)

In [ ]:
oil_prices.tail(5)

In [ ]:
oil_prices.shape

In [ ]:
full_oil_prices=oil_prices  # Saving the 'full' dataset as 'full_oil_prices' which will include the 2015 year

full_oil_prices.head(5)

In [ ]:
mask_2015=oil_prices['TIME'].str.contains('2015')  # Isolating 2015 to be used as test data later

test_data=oil_prices[mask_2015]

type(test_data)



In [ ]:
print(test_data) # Counting number of rows so know how many to remove from training set

In [ ]:
oil_prices=oil_prices.iloc[:-168]   # Removing the last 2015 data so it can become a test set

oil_prices.tail(5)

In [ ]:
non_scaled_oil_prices=oil_prices.iloc[:, 2].values   # Create numpy array from 'Value' column for keras module

type(non_scaled_oil_prices) # Checking it's the correct format

## Min-Max Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler # Importing min-max scaler class for min-max scaling

In [ ]:
scaler=MinMaxScaler() # Creating instance of the min-max scaler class

In [ ]:
min_max_oil_prices=scaler.fit_transform(non_scaled_oil_prices.reshape(-1, 1)) # Performing min-max scaling on numpy array

In [ ]:
min_max_oil_prices.shape # len(min_max_oil_prices)

In [ ]:
x_training_data=[]        # Initialising two data structures for the timesteps
y_training_data=[] 

In [ ]:
for i in range(50, len(min_max_oil_prices)):               # Populating the data structures for the timesteps
               x_training_data.append(min_max_oil_prices[i-50:i, 0])
               y_training_data.append(min_max_oil_prices[i, 0])

In [ ]:
x_training_data=np.array(x_training_data)              # Converting to numpy arrays
y_training_data=np.array(y_training_data)

print(x_training_data.shape)

In [ ]:
print(y_training_data.shape)

In [ ]:
print(len(min_max_oil_prices)-50)  # Confirm arrays have correct shape, len(min_max_oil_prices) - 50 = 5602

In [ ]:
x_training_data=np.reshape(x_training_data, (x_training_data.shape[0], x_training_data.shape[1], 1)) 

print(x_training_data.shape)    # Turning numpy array 3 dimensional for tensorflow

In [ ]:
from tensorflow.keras.models import Sequential   # Classes for neural network building
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [ ]:
rnn=Sequential()  # Initialising RNN

In [ ]:
rnn.add(LSTM(units=5, return_sequences=True, input_shape=(x_training_data.shape[1], 1))) # First LSTM layer with flexible timestep approach

In [ ]:
rnn.add(Dropout(0.2)) # Including 20% dropout rate as is convention

In [ ]:
for i in [True, False]:            
    rnn.add(LSTM(units=5, return_sequences=i))
    rnn.add(Dropout(0.2))                            # Adding two more LSTM layers

In [ ]:
rnn.add(Dense(units=1))   # Output layer

In [ ]:
rnn.compile(optimizer='adam', loss='mean_squared_error') # optimizer and loss function

In [ ]:
rnn.fit(x_training_data, y_training_data, epochs=10, batch_size=10)  # Fitting model to training data

In [ ]:
plt.plot(test_data_min_max)           # Plotting test data

In [ ]:
all_unscaled_min_max_data=pd.concat((oil_prices['Value'], test_data['Value']), axis=0) # Concatenate training and test data

In [ ]:
x_test_data=all_unscaled_min_max_data[len(all_unscaled_min_max_data)-len(test_data)-50:].values

In [ ]:
len(x_test_data)      # 168 test data + 50 time steps = 218 shows the array is the right length

In [ ]:
x_test_data=np.reshape(x_test_data, (-1, 1)) # Reshaping to make suitable for predict method

In [ ]:
x_test_data_scaled=scaler.transform(x_test_data)  # Scaling x_test_data 

In [ ]:
final_x_test_data=[]
for i in range(50, len(x_test_data_scaled)):
    final_x_test_data.append(x_test_data_scaled[i-50:i, 0])
final_x_test_data=np.array(final_x_test_data)

In [ ]:
final_x_test_data=np.reshape(final_x_test_data, (final_x_test_data.shape[0], final_x_test_data.shape[1], 1)) # Making 3D for tensorflow

In [ ]:
min_max_predictions=rnn.predict(final_x_test_data)   # Making the prediction with min_max approach

In [ ]:
plt.plot(min_max_predictions) # Scaled predictions with min-max

In [ ]:
unscaled_min_max_predictions=scaler.inverse_transform(min_max_predictions)  # Unscaling for actual values

plt.plot(unscaled_min_max_predictions)

In [ ]:
#test_data=test_data.values



In [ ]:
from sklearn.metrics import mean_squared_error

min_max_mse=mean_squared_error(test_data, unscaled_min_max_predictions)

## Add next steps here from first approach

## Z-Score scaling

In [ ]:
from sklearn import preprocessing    # For creating a standard scaler

scaler=preprocessing.StandardScaler()  # Creating standard scaler

In [ ]:
z_score_oil_prices=scaler.fit_transform(non_scaled_oil_prices.reshape(-1, 1))  # Performing z-score scaling on numpy array


In [ ]:
z_score_oil_prices.shape # len(z_score_oil_prices)

In [ ]:
x_training_data=[]        # Initialising two data structures for the timesteps
y_training_data=[] 

In [ ]:
for i in range(50, len(z_score_oil_prices)):               # Populating the data structures for the timesteps
               x_training_data.append(z_score_oil_prices[i-50:i, 0])
               y_training_data.append(z_score_oil_prices[i, 0])

In [ ]:
x_training_data=np.array(x_training_data)              # Converting to numpy arrays
y_training_data=np.array(y_training_data)

print(x_training_data.shape)

In [ ]:
print(y_training_data.shape)

In [ ]:
print(len(z_score_oil_prices)-50)  # Confirm arrays have correct shape, len(min_max_oil_prices) - 50 = 5770

In [ ]:
x_training_data=np.reshape(x_training_data, (x_training_data.shape[0], x_training_data.shape[1], 1)) 

print(x_training_data.shape)    # Turning numpy array 3 dimensional for tensorflow

In [ ]:
rnn=Sequential()  # Initialising RNN

In [ ]:
rnn.add(LSTM(units=5, return_sequences=True, input_shape=(x_training_data.shape[1], 1))) # Replicating same parameters as min-max scaling round

In [ ]:
rnn.add(Dropout(0.2)) # Replicating same dropout rate as min-max scaling round

In [ ]:
for i in [True, False]:            
    rnn.add(LSTM(units=5, return_sequences=i))
    rnn.add(Dropout(0.2))                            # Replicating number of LSTM layers with min-max round

In [ ]:
rnn.add(Dense(units=1))   # Identical output layer as min-max round

In [ ]:
rnn.compile(optimizer='adam', loss='mean_squared_error') # Same optimizer and loss function as min-max round

In [ ]:
#rnn.fit(x_training_data, y_training_data, epochs=10, batch_size=10)  # Fitting model to z-score scaled training data